# Sequential Streaming - Function-Based Pattern

## Overview
This notebook demonstrates how to build sequential workflows using **function-based executors** with the `@executor` decorator. You'll learn how to:
- Use `@executor` decorator to create lightweight executors from functions
- Build the same transformation pipeline as `sequential_executors.ipynb` with less boilerplate
- Stream workflow events for observability
- Compare class-based vs. function-based executor patterns

## What This Sample Does
The workflow performs the same text transformation pipeline as the class-based version:
1. **to_upper_case** - Converts input text to uppercase
2. **reverse_text** - Reverses the uppercase text

**Input:** `"hello world"`  
**Output:** `"DLROW OLLEH"`

## Key Differences from Class-Based Pattern
- **Less Boilerplate:** No need to create classes or `__init__` methods
- **Functional Style:** Simple functions decorated with `@executor`
- **Same Capabilities:** Full access to `WorkflowContext` and all framework features
- **Best for:** Stateless transformations and simple logic

## Step 1: Import Required Libraries

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio

import os
from dotenv import load_dotenv
from agent_framework import (
    ExecutorCompletedEvent,
    ExecutorInvokedEvent,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    executor,
)
from typing_extensions import Never
# Load environment variables from .env file
load_dotenv('../../.env')


## Step 2: Define Function-Based Executors
Create executors using the `@executor` decorator on async functions.

**Key Points:**
- `@executor(id=...)` decorator converts functions into workflow executors
- No need for `@handler` - the function itself is the handler
- Same `WorkflowContext` typing as class-based executors
- Much less code than class-based pattern

In [ ]:
@executor(id="to_upper_case")
async def to_upper_case(text: str, ctx: WorkflowContext[str]) -> None:
    """Convert text to uppercase and send to next executor."""
    upper_text = text.upper()
    await ctx.send_message(upper_text)


@executor(id="reverse_text")
async def reverse_text(text: str, ctx: WorkflowContext[Never, str]) -> None:
    """Reverse text and yield as final output."""
    reversed_text = text[::-1]
    await ctx.yield_output(reversed_text)

## Step 3: Build and Run the Workflow
The workflow construction is identical to the class-based version.

**Workflow Structure:**
```
Input → to_upper_case → reverse_text → Output
```

In [ ]:
async def run_workflow():
    """Build and execute the sequential workflow using function-based executors."""
    # Build the workflow - functions are used directly as executors
    workflow = (
        WorkflowBuilder()
        .set_start_executor(to_upper_case)
        .add_edge(to_upper_case, reverse_text)
        .build()
    )

    # Run with streaming for full observability
    input_text = "hello world"
    print(f"Input: {input_text}")
    print("\n=== Workflow Events ===")
    
    async for event in workflow.run_stream(input_text):
        if isinstance(event, ExecutorInvokedEvent):
            print(f"▶ Executor Invoked: {event.executor_id}")
        elif isinstance(event, ExecutorCompletedEvent):
            print(f"✓ Executor Completed: {event.executor_id}")
        elif isinstance(event, WorkflowOutputEvent):
            print(f"\n=== Final Output ===")
            print(f"Result: {event.data}")

## Step 4: Execute the Workflow

In [ ]:
await run_workflow()

## Expected Output
```
Input: hello world

=== Workflow Events ===
▶ Executor Invoked: to_upper_case
✓ Executor Completed: to_upper_case
▶ Executor Invoked: reverse_text
✓ Executor Completed: reverse_text

=== Final Output ===
Result: DLROW OLLEH
```

## Pattern Comparison: Class-Based vs. Function-Based

### Class-Based Executors
```python
class UpperCaseExecutor(Executor):
    def __init__(self, id: str | None = None):
        super().__init__(id=id or "upper_case_executor")
    
    @handler
    async def to_upper_case(self, text: str, ctx: WorkflowContext[str]) -> None:
        upper_text = text.upper()
        await ctx.send_message(upper_text)
```

### Function-Based Executors
```python
@executor(id="to_upper_case")
async def to_upper_case(text: str, ctx: WorkflowContext[str]) -> None:
    upper_text = text.upper()
    await ctx.send_message(upper_text)
```

**Lines of code:** ~8 lines → ~4 lines (50% reduction)

## When to Use Each Pattern

### Use Function-Based (`@executor`) When:
✅ Executor logic is stateless (no instance variables)  
✅ Simple transformations or data processing  
✅ You want minimal boilerplate  
✅ Single-purpose executors  
✅ Quick prototyping or scripts

### Use Class-Based (`Executor`) When:
✅ Need to maintain state across invocations  
✅ Require initialization parameters  
✅ Multiple handler methods or helper functions  
✅ Complex business logic  
✅ Reusable components across workflows  
✅ Need lifecycle methods (setup, teardown)

## Key Takeaways

### Function-Based Advantages
- **Conciseness:** 50% less code for simple executors
- **Readability:** Clear intent without class structure overhead
- **Simplicity:** No need to understand class inheritance
- **Same Power:** Full access to `WorkflowContext` and all framework features

### Execution Flow
The execution flow is **identical** to the class-based version:
1. Input `"hello world"` → `to_upper_case`
2. `to_upper_case` transforms to `"HELLO WORLD"` → sends message
3. `reverse_text` receives `"HELLO WORLD"` → reverses to `"DLROW OLLEH"`
4. `reverse_text` yields final output

### Event Streaming
Both patterns support the same event streaming capabilities:
- `ExecutorInvokedEvent` - Executor starts
- `ExecutorCompletedEvent` - Executor finishes
- `WorkflowOutputEvent` - Final result

## Best Practices
- Start with function-based executors for simplicity
- Migrate to class-based when you need state or complexity
- Mix both patterns in the same workflow (they're fully compatible)
- Use meaningful executor IDs for debugging

## Next Steps
- Compare with `sequential_executors.ipynb` to see the class-based version
- Explore `edge_condition.ipynb` for conditional routing
- See `multi_selection_edge_group.ipynb` for parallel branching